# Linear Regression Weather Forecast Model
This notebook trains a linear regression model to predict average temperature using Hanoi 10-year weather data.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import joblib

# Load data
df = pd.read_csv("Hanoi_temp_10_years.csv")

# Feature engineering
df['datetime'] = pd.to_datetime(df['datetime'])
df['month'] = df['datetime'].dt.month
df['dayofyear'] = df['datetime'].dt.dayofyear
df['sin_doy'] = np.sin(2 * np.pi * df['dayofyear'] / 365)
df['cos_doy'] = np.cos(2 * np.pi * df['dayofyear'] / 365)

# Define features and target
X = df[['tempmax','tempmin','feelslikemax','feelslikemin','feelslike','dew','humidity',
        'precip','windspeed','windgust','winddir','sealevelpressure','cloudcover',
        'visibility','solarradiation','solarenergy','uvindex','month','dayofyear',
        'sin_doy','cos_doy']]
y = df['temp']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

metrics_train = {
    'r2': r2_score(y_train, y_pred_train),
    'rmse': np.sqrt(mean_squared_error(y_train, y_pred_train)),
    'mae': mean_absolute_error(y_train, y_pred_train)
}
metrics_test = {
    'r2': r2_score(y_test, y_pred_test),
    'rmse': np.sqrt(mean_squared_error(y_test, y_pred_test)),
    'mae': mean_absolute_error(y_test, y_pred_test)
}

print("Train metrics:", metrics_train)
print("Test metrics:", metrics_test)

# Save model
joblib.dump(model, "weather_lr_model.pkl")

# Save predictions
pred_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_test})
pred_df.to_csv("predictions.csv", index=False)

# Plot
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred_test, alpha=0.5)
plt.xlabel("Actual Temperature")
plt.ylabel("Predicted Temperature")
plt.title("Predicted vs Actual Temperature")
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')
plt.savefig("pred_vs_actual.png")
plt.show()
